In [1]:
# !pip install empyrical-reloaded

In [2]:
import numpy as np
import pandas as pd
import empyrical as ep

## リターン計算

In [3]:
arr_returns = np.array(
    [
        [.01, .01, .02, -.04, -.06, -.02, -.03],
        [.03, .02, .01, -.01, -.02, -.03, -.04]
    ]
).T
arr_returns

array([[ 0.01,  0.03],
       [ 0.01,  0.02],
       [ 0.02,  0.01],
       [-0.04, -0.01],
       [-0.06, -0.02],
       [-0.02, -0.03],
       [-0.03, -0.04]])

In [4]:
arr_returns.shape

(7, 2)

In [5]:
# 日次リターン -> 累積リターン
arr_cum_returns = ep.cum_returns(returns=arr_returns)
arr_cum_returns

array([[ 0.01      ,  0.03      ],
       [ 0.0201    ,  0.0506    ],
       [ 0.040502  ,  0.061106  ],
       [-0.00111808,  0.05049494],
       [-0.061051  ,  0.02948504],
       [-0.07982998, -0.00139951],
       [-0.10743508, -0.04134353]])

In [6]:
# 累積リターン -> 株価
arr_prices = 1 + np.pad(array=arr_cum_returns, pad_width=((1, 0), (0, 0)))
arr_prices

array([[1.        , 1.        ],
       [1.01      , 1.03      ],
       [1.0201    , 1.0506    ],
       [1.040502  , 1.061106  ],
       [0.99888192, 1.05049494],
       [0.938949  , 1.02948504],
       [0.92017002, 0.99860049],
       [0.89256492, 0.95865647]])

In [7]:
# 株価 -> 日次リターン
ep.simple_returns(arr_prices)

array([[ 0.01,  0.03],
       [ 0.01,  0.02],
       [ 0.02,  0.01],
       [-0.04, -0.01],
       [-0.06, -0.02],
       [-0.02, -0.03],
       [-0.03, -0.04]])

In [8]:
# 日次リターン -> 年率リターン
ep.annual_return(
    returns=arr_returns,
    period='daily',
#     annualization=252
)

array([-0.98328781, -0.78129018])

In [9]:
# 計算結果確認
arr_prices[-1]**(252/(arr_prices.shape[0]-1)) - 1

array([-0.98328781, -0.78129018])

## 各指標計算

In [10]:
arr_returns = np.array(
    [
        [.01, .01, .02, -.04, -.06, -.02, -.03],
        [.03, .02, .01, -.01, -.02, -.03, -.04]
    ]
).T
arr_returns

array([[ 0.01,  0.03],
       [ 0.01,  0.02],
       [ 0.02,  0.01],
       [-0.04, -0.01],
       [-0.06, -0.02],
       [-0.02, -0.03],
       [-0.03, -0.04]])

### sharpe ratio

In [11]:
# calculate the sharpe ratio
ep.sharpe_ratio(returns=arr_returns, risk_free=0, period='daily')

array([-8.33730402, -3.44029307])

In [12]:
# 計算結果確認
np.nanmean(arr_returns, axis=0) / np.nanstd(arr_returns, ddof=1, axis=0) * (252**0.5)

array([-8.33730402, -3.44029307])

In [13]:
# calculate the rolling sharpe ratio
# window日間のリターン・リスクでsharpe ratioを計算
# shape: (T-window+1)
ep.roll_sharpe_ratio(arr=arr_returns[:, 0], window=3)

array([ 36.66060556,  -1.64610978, -10.16782255, -31.74901573,
       -27.96151201])

### drawdown, calmar ratio

In [14]:
# calculate the max drawdown
ep.max_drawdown(returns=arr_returns)

array([-0.14217856, -0.09654976])

In [15]:
# calculate the calmar ratio
ep.calmar_ratio(returns=arr_returns[:, 0], period='daily')

-6.915865582405438

In [16]:
# 計算結果確認
ep.annual_return(returns=arr_returns) / np.abs(ep.max_drawdown(returns=arr_returns))

array([-6.91586558, -8.09209863])

In [17]:
# calculate the rolling max drawdown
ep.roll_max_drawdown(arr=arr_returns[:, 0], window=3)

array([ 0.      , -0.04    , -0.0976  , -0.115648, -0.106436])

### downside risk

In [18]:
# 下落リターンのみを対象としたリスク指標
ep.downside_risk(returns=arr_returns)

array([0.48373546, 0.32863353])

In [19]:
# 計算結果確認
downside_diff = np.clip(arr_returns, np.NINF, 0)
downside_diff = np.square(downside_diff)
(np.nanmean(downside_diff, axis=0) * 252)**0.5

array([0.48373546, 0.32863353])

In [20]:
# calculate the sortino ratio
ep.sortino_ratio(returns=arr_returns, required_return=0, period='daily')

array([-8.18629248, -4.38178046])

In [21]:
# 計算結果確認
np.nanmean(arr_returns, axis=0) / ep.downside_risk(returns=arr_returns) * 252

array([-8.18629248, -4.38178046])

### VaR, CVaR

In [22]:
ep.value_at_risk(returns=arr_returns[:, 0], cutoff=0.05)

-0.054

In [23]:
# 計算結果確認
np.percentile(arr_returns[:, 0], 100 * 0.05)

-0.054

In [24]:
ep.conditional_value_at_risk(returns=arr_returns[:, 0], cutoff=0.05)

-0.06

In [25]:
# 計算結果確認
cutoff_index = int((len(arr_returns[:, 0]) - 1) * 0.05)
np.mean(np.partition(arr_returns[:, 0], cutoff_index)[: cutoff_index + 1])

-0.06

### alpha, beta

In [26]:
arr_factor_returns = np.array(
    [
        [.02, .02, .03, -.05, -.05, -.01, -.03],
        [.03, .02, .01, -.01, -.02, -.03, -.03]
    ]
).T

In [27]:
# calculate alpha and beta
alpha, beta = ep.alpha_beta(returns=arr_returns, factor_returns=arr_factor_returns)
print(alpha)
print(beta)

[-0.83576539  0.85714286]
[-0.24602659  1.072     ]


## データ整形

In [28]:
# !pip install nasdaqdatalink

In [29]:
import nasdaqdatalink

In [30]:
df = nasdaqdatalink.get("USTREASURY/YIELD")

In [31]:
df_returns = ep.simple_returns(df)
df_returns

,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
1990-01-03,NaN,NaN,0.007663,0.006337,0.005122,0.008895,0.007595,0.006353,0.007519,0.006297,NaN,0.005000
1990-01-04,NaN,NaN,-0.006337,-0.005038,-0.003822,-0.002519,-0.003769,-0.001263,-0.002488,-0.001252,NaN,0.000000
1990-01-05,NaN,NaN,-0.006378,-0.006329,-0.003836,-0.002525,0.001261,0.001264,0.001247,0.001253,NaN,0.002488
1990-01-08,NaN,NaN,0.000000,0.003822,0.002567,0.000000,0.001259,0.000000,0.002491,0.003755,NaN,0.003722
1990-01-09,NaN,NaN,0.001284,-0.007614,-0.003841,0.001266,-0.001258,0.000000,0.000000,0.000000,NaN,0.001236
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31,-0.250000,0.300000,0.157895,0.139535,0.040000,0.026087,0.007246,0.006211,0.005747,0.005618,0.014019,0.019324
2022-02-01,0.333333,-0.230769,-0.136364,-0.020408,0.000000,0.000000,0.000000,0.006173,0.005714,0.011173,0.009217,0.004739
2022-02-02,0.000000,0.000000,0.000000,-0.062500,-0.025641,-0.016949,-0.007194,-0.018405,-0.011364,-0.016575,-0.009132,-0.004717


In [32]:
monthly_ret_table = ep.aggregate_returns(returns=df_returns[["3 MO", "10 YR"]], convert_to="monthly")
monthly_ret_table

3 MO     10 YR
Date Date                        
1990 1     2.171137e-02  0.061713
     2     5.000000e-03  0.009490
     3     3.731343e-03  0.016451
     4     2.220446e-16  0.045087
     5    -7.434944e-03 -0.048673
...                 ...       ...
2021 10    2.500000e-01  0.019737
     11   -2.220446e-16 -0.077419
     12    2.000000e-01  0.062937
2022 1     2.666667e+00  0.177632
     2     4.545455e-02  0.078212

[386 rows x 2 columns]

In [33]:
monthly_ret_table = monthly_ret_table.unstack()
monthly_ret_table

3 MO                                                          \
Date            1             2             3             4             5    
Date                                                                         
1990  2.171137e-02  5.000000e-03  3.731343e-03  2.220446e-16 -7.434944e-03   
1991 -3.921569e-02 -2.354788e-02 -4.823151e-02 -4.054054e-02  5.281690e-03   
1992 -5.050505e-03  2.284264e-02  2.977667e-02 -8.674699e-02  4.440892e-16   
1993 -6.031746e-02  1.689189e-02 -1.993355e-02  6.779661e-03  5.387205e-02   
1994 -6.514658e-03  1.377049e-01  2.593660e-02  1.151685e-01  8.564232e-02   
1995  5.633803e-02 -1.000000e-02 -1.010101e-02 -1.700680e-03 -1.022147e-02   
1996 -9.803922e-03 -5.940594e-03  2.191235e-02  1.949318e-03  7.782101e-03   
1997 -1.151631e-02  1.359223e-02  2.490421e-02 -1.308411e-02 -6.060606e-02   
1998 -3.171642e-02  2.504817e-02 -3.007519e-02 -3.100775e-02  6.000000e-03   
1999  0.000000e+00  4.017857e-02 -3.648069e-02  1.336303e-02  2.417582e-02   
2000  8.067542e-02  3.472222e-03  1.730104e-02 -1.020408e-02 -3.264605e-02   
2001 -1.528014e-01 -2.805611e-02 -1.134021e-01 -8.139535e-02 -8.101266e-02   
2002  1.149425e-02  1.704545e-02 -1.110223e-16 -1.117318e-02 -1.694915e-02   
2003 -3.278689e-02  1.694915e-02 -5.000000e-02 -8.771930e-03 -1.769912e-02   
2004 -3.157895e-02  4.347826e-02 -1.041667e-02  3.157895e-02  1.020408e-01   
2005  1.306306e-01  9.960159e-02  1.086957e-02  3.942652e-02  3.103448e-02   
2006  9.558824e-02  3.355705e-02  2.164502e-03  3.023758e-02  1.886792e-02   
2007  1.992032e-02  7.812500e-03 -2.325581e-02 -2.579365e-02 -3.665988e-02   
2008 -4.166667e-01 -5.612245e-02 -2.540541e-01  3.623188e-02  3.216783e-01   
2009  1.181818e+00  8.333333e-02 -1.923077e-01 -3.333333e-01 -1.110223e-16   
2010  3.333333e-01  6.250000e-01  2.307692e-01  0.000000e+00  0.000000e+00   
2011  2.500000e-01  0.000000e+00 -4.000000e-01 -5.555556e-01  5.000000e-01   
2012  2.000000e+00  3.333333e-01 -1.250000e-01  4.285714e-01 -3.000000e-01   
2013  4.000000e-01  5.714286e-01 -3.636364e-01 -2.857143e-01 -2.000000e-01   
2014 -7.142857e-01  1.500000e+00 -1.110223e-16 -4.000000e-01  3.333333e-01   
2015 -5.000000e-01  0.000000e+00  5.000000e-01 -6.666667e-01  0.000000e+00   
2016  1.062500e+00  2.220446e-16 -3.636364e-01  4.761905e-02  5.454545e-01   
2017  1.960784e-02  1.923077e-02  4.339623e-01           NaN  2.250000e-01   
2018  5.035971e-02  1.301370e-01  4.848485e-02  8.092486e-02  3.208556e-02   
2019 -1.632653e-02  1.659751e-02 -2.040816e-02  1.250000e-02 -3.292181e-02   
2020 -2.220446e-16 -1.806452e-01           NaN -1.818182e-01  5.555556e-01   
2021 -3.333333e-01 -3.333333e-01 -2.500000e-01 -6.666667e-01  0.000000e+00   
2022  2.666667e+00  4.545455e-02           NaN           NaN           NaN   

                                                                ...  \
Date            6             7         8         9         10  ...   
Date                                                            ...   
1990 -1.248439e-03 -3.250000e-02 -0.014212 -0.034076 -0.004071  ...   
1991  0.000000e+00 -1.751313e-03 -0.036842 -0.041894 -0.057034  ...   
1992 -3.693931e-02 -1.095890e-01 -0.006154 -0.148607  0.101818  ...   
1993 -9.584665e-03  0.000000e+00 -0.006452 -0.032468  0.040268  ...   
1994 -1.160093e-02  3.051643e-02  0.066059  0.025641  0.083333  ...   
1995 -3.614458e-02 -1.110223e-16 -0.026786 -0.009174  0.014815  ...   
1996  2.220446e-16  2.702703e-02 -0.005639 -0.028355  0.005837  ...   
1997  5.846774e-02 -3.330669e-16 -0.001905 -0.034351  0.029644  ...   
1998  1.391650e-02 -3.330669e-16 -0.027451 -0.118952 -0.009153  ...   
1999  2.575107e-02 -6.276151e-03  0.048421 -0.020080  0.049180  ...   
2000  4.440497e-02  6.632653e-02  0.006380 -0.012678  0.024077  ...   
2001  5.509642e-03 -3.013699e-02 -0.048023 -0.287834 -0.145833  ...   
2002 -2.298851e-02  5.882353e-03 -0.011696 -0.071006 -0.082803  ...   
2003 -1.891892e-01  6.666667e-02  0.020833 -0.030612  0.010526  ...  